In [70]:
import gzip
from collections import defaultdict
import sklearn
from sklearn import linear_model
import string
import re
import datetime, time
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as md
import math

In [71]:
def readCSV(path):
    f = open(path, 'rt')
    f.readline()

    for l in f:
        
        if l.startswith("timestamp"): 
            continue

        try: 
            timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,stockgrantvalue,bonus,gender,otherdetails,cityid,dmaid,rowNumber,Masters_Degree,Bachelors_Degree,Doctorate_Degree,Highschool,Some_College,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education = re.split(r',(?![ ])', l)

        except:
            continue

        yield timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,stockgrantvalue,bonus,gender,otherdetails,cityid,dmaid,rowNumber,Masters_Degree,Bachelors_Degree,Doctorate_Degree,Highschool,Some_College,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education

In [72]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

In [73]:
def Cosine(x1,x2):
    numer = 0
    norm1 = 0
    norm2 = 0
    for a1,a2 in zip(x1,x2):
        numer += a1*a2
        norm1 += a1**2
        norm2 += a2**2
    if norm1*norm2:
        return numer / math.sqrt(norm1*norm2)
    return 0

In [74]:
#####
# Parse data from dataset
#####

In [75]:
# id , unixtime, company, level, title, total_comp, city, state, experience, tenure, tag, 
# salary, stock, bonus, city_id, dma_id, ms_deg, bs_deg, phd_deg, hs, college
def parseSalaryRaw(salaryRaw):
    salaryAllData = []

    for entry in salaryRaw:

        date, clocktime = entry[0].split(' ')
        date = date.split('/')
        clocktime = clocktime.split(':')
        timestamp = datetime.datetime(int(date[2]), int(date[0]), int(date[1]), int(clocktime[0]), int(clocktime[1]), int(clocktime[2]))
        timestamp = int(time.mktime(timestamp.timetuple()))


        company = entry[1]
        level = entry[2]
        title = entry[3]

        total_comp = int(float(entry[4]))

        location = entry[5]
        if location.count(',') == 2:
            city, state, country = location.strip('"').split(', ')
        elif location.count(',') == 1:
            city, state = location.strip('"').split(', ')
            country = 'USA'

        experience = entry[6]
        tenure = entry[7]

        tag = entry[8]

        salary = int(float(entry[9]))
        if salary == 0:
            continue

        stock = int(float(entry[10]))
        bonus = int(float(entry[11]))

        g = entry[12]
        gender = -1
        if g == 'Female':
            gender = 0
        elif g == 'Male':
            gender = 1
        elif g == 'Other':
            gender = 2

        city_id = int(entry[14])

        try: 
            dma_id = int(entry[15])
        except:
            dma_id = -1

        id = int(entry[16])

        ms_deg = int(entry[17])
        bs_deg = int(entry[18])
        phd_deg = int(entry[18])
        hs = int(entry[19])
        college = int(entry[20])

        r = entry[27]
        race = -1
        if r == 'Asian':
            race = 0
        elif r == 'Black':
            race = 1
        elif r == 'Hispanic':
            race = 2
        elif r == 'Two Or More':
            race = 3
        elif r == 'White':
            race = 4


        salaryAllData.append({
            'id': id,
            'timestamp': timestamp,
            'company': company,
            'level': level,
            'title': title,
            'total_comp': total_comp,
            'city': city,
            'state': state,
            'country': country,
            'experience': experience,
            'tenure': tenure,
            'tag': tag,
            'salary': salary,
            'stock': stock,
            'bonus': bonus,
            'gender': gender,
            'city_id': city_id,
            'dma_id': dma_id,
            'ms_deg': ms_deg,
            'bs_deg': bs_deg,
            'phd_deg': phd_deg,
            'hs': hs,
            'college': college,
            'race': race,
        })
    return salaryAllData


In [76]:
# Reparse data from CSV so we don't mess up above data
salaryRaw = []
for l in readCSV("Levels_Fyi_Salary_Data.csv"):
    salaryRaw.append(l)

salaryAllData = parseSalaryRaw(salaryRaw)
salaryAllData[0]['id']

1

In [77]:
# Parse raw data
salaryAllData = parseSalaryRaw(salaryRaw)

In [78]:
#####
# Training, Validation, and Test sets
#####

# shuffle data
shuffledAllData = sklearn.utils.shuffle(salaryAllData)
shuffledAllData[0]['id']

# create 80 / 10 / 10 : Train / Validation / Test sets
X = shuffledAllData
y = [d['salary'] for d in shuffledAllData]

X_train, X_temp, y_train, y_temp = sklearn.model_selection.train_test_split(X, y, test_size=0.2, random_state=1)
X_valid, X_test, y_valid, y_test = sklearn.model_selection.train_test_split(X_temp, y_temp, test_size=0.5, random_state=1)

# Validate that sets were created with propper sizes
print(str(len(X_train)) + ", " + str(len(y_train)))
print(str(len(X_valid)) + ", " + str(len(y_valid)))
print(str(len(X_test)) + ", " + str(len(y_test)))

# Validate all ids are different
print(str(X_train[0]['id']) + ", " + str(X_valid[0]['id']) + ", " + str(X_test[0]['id']))

47996, 47996
6000, 6000
6000, 6000
23169, 29382, 80711


In [79]:
# Create some useful collections for these baseline

# create YOE to Average Map based on training data
# We create two dictionaries to track the total salary seen at each YOE
salByExpMap = defaultdict(int)

# And the number of salaries seen at that YOE
entriesByExpMap = defaultdict(int)

# For each entry increment the total sal and num entries at that YOE
# (This could be done with a running avg technique but that is too much work to code)
for x in X_train:
    exp, sal = float(x['experience']), float(x['salary'])
    exp = math.ceil(exp)
    
    total = 0
    numEntries = 0
    
    if salByExpMap[exp]:
        total = salByExpMap[exp]
    if entriesByExpMap[exp]:
        numEntries = entriesByExpMap[exp]
    
    total += sal
    numEntries += 1
    
    salByExpMap[exp] = total
    entriesByExpMap[exp] = numEntries
    

    
# Calculate averages by YOE as well as total avg
totalSal = 0
totalCount = 0

averageSalaryByYoeMap = defaultdict(int)
for exp in entriesByExpMap:
    avg = salByExpMap[exp] / entriesByExpMap[exp]
    averageSalaryByYoeMap[exp] = avg
    
    totalSal += salByExpMap[exp]
    totalCount += entriesByExpMap[exp]

avgSal = totalSal / totalCount

print("Average salary: " + str(avgSal))


def pred_baseline1(x):    
    return avgSal
    
def pred_baseline2(x):    
    exp = math.ceil(float(x['experience']))

    if averageSalaryByYoeMap[exp]:
        return averageSalaryByYoeMap[exp]
    else:
        return avgSal

Average salary: 142082.909575798


In [80]:
def baseline1():
    # Returns average salary in dataset
    y_b1 = []
    y_b1_pred = []
    
    avg_err = 0
    avg_percent_err = 0
    
    for d in X_valid:
        pred = pred_baseline1(d)
        actual = d['salary']
        
        y_b1_pred.append(pred)
        y_b1.append(actual)
        
        avg_err += abs(actual - pred)
        avg_percent_err += abs(100 * ((actual - pred) / actual))
        
    mse = MSE(y_b1_pred, y_b1)
    
    avg_err = avg_err / len(X_valid)
    avg_percent_err = avg_percent_err / len(X_valid)
    
    return mse, avg_err, avg_percent_err

In [81]:
mse_b1, avg_err_b1, avg_percent_err_b1 = baseline1()
print(mse_b1)
print(avg_err_b1)
print(avg_percent_err_b1)

3104093944.9357805
39999.378042586904
51.309502627271016


In [82]:
def baseline2():
    # Takes rounded YOE for each datapoint and predicts average salary for that YOE
    # If there are no entries for that YOE, predict overall average
    y_b1 = []
    y_b1_pred = []
    
    avg_err = 0
    avg_percent_err = 0
    
    for d in X_valid:
        pred = pred_baseline2(d)
        actual = d['salary']
        
        y_b1_pred.append(pred)
        y_b1.append(actual)
        
        avg_err += abs(actual - pred)
        avg_percent_err += abs(100 * ((actual - pred) / actual))
        
    mse = MSE(y_b1_pred, y_b1)
    
    avg_err = avg_err / len(X_valid)
    avg_percent_err = avg_percent_err / len(X_valid)
    
    return mse, avg_err, avg_percent_err

In [83]:
mse_b2, avg_err_b2, avg_percent_err_b2 = baseline2()
print(mse_b2)
print(avg_err_b2)
print(avg_percent_err_b2)

2506204817.606467
35020.29687154801
44.97166973802387


In [84]:
#####
# COMBINED MODEL
#####

In [85]:
y_train = [d['salary'] for d in X_train]

In [86]:
# Create BOW feature

In [87]:
# Inspired by and midified from workbook chapter 8

dSize = 500
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in X_train:
    r = d['company'] + ' ' + d['level'] + ' ' + d['title'] + ' ' + d['tag']
    r = r.lower()
#     r = ''.join([c for c in r if not c in punctuation])
    for w in r.split():
        wordCount[w] += 1

        
counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

words = [x[1] for x in counts[:dSize]]

wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

In [88]:
def featureBOW(d):
    feat = [0]*len(words)
    r = d['company'] + ' ' + d['level'] + ' ' + d['title'] + ' ' + d['tag']
    r = r.lower()
#     r = ''.join([c for c in r if not c in punctuation])
    
    for w in r.split():
        if w in words:
            feat[wordId[w]] += 1
    feat.append(float(d['experience'])) # offset
    return feat

In [89]:
# Create DMA feature

In [90]:
#counting all dma_id just for exploration
#assigning indices for every dma_id
def dma_count_plus_index(dict, ind_dict):
    count = 1
    for d in salaryRaw:
        dict[d[15]] += 1
        if not ind_dict.get(d[15]):
            ind_dict[d[15]] = count
            count += 1
        

In [91]:
dmaidcounts = defaultdict(int)
dmaindices = {}
dma_count_plus_index(dmaidcounts, dmaindices)

In [92]:
def dma_yoe_feat(d, dict): #returns row for dma_id and either tenure/yoe
    # print(exp, type(exp))

    exp = float(d['experience'])
    

    id = str(d['dma_id'])

    arr = [1] + ([0]*149)
    if id == '-1':
         arr[77] = 1
    elif id != '807':
        arr[(dict[id]-1)] = 1
    return arr + [exp]

In [93]:
# Create yoe/tenure/edication feature

In [94]:
def feat_deg(d):
    exp = float(d['experience'])
    ten = float(d['tenure'])
    if d['hs'] == 1:
        edu = 12.0
    elif d['college'] == 1:
        edu = 14.0
    elif d['bs_deg'] == 1:
        edu = 16.0
    elif d['ms_deg'] == 1:
        edu = 18.0
    elif d['phd_deg'] == 1:
        edu = 21.0
    else:
        edu = 16.0
    return [exp, ten, edu]

In [95]:
# create time feature

In [96]:
minYear = 2017
maxYear = 2021

def feature_time(d, inc_year=True, inc_month=True, inc_day=True, inc_hour=True):
    
    unix_time = d['timestamp']
    date = datetime.datetime.fromtimestamp(unix_time)

    year = date.year
    month = date.month
    day = date.day
    hour = date.hour
    minute = date.minute

    one_hot_year = [0]*(maxYear-minYear)
    y_index = year - minYear

    if y_index != 0:
        one_hot_year[y_index-1] = 1

    one_hot_month = [0]*11
    m_index = month - 1

    if m_index != 0:
        one_hot_month[m_index-1] = 1

    one_hot_day = [0]*31
    d_index = day - 1

    if d_index != 0:
        one_hot_day[d_index-1] = 1

    one_hot_hour = [0]*23
    h_index = hour - 1

    if h_index != 0:
        one_hot_hour[h_index-1] = 1

    yoe = float(d['experience'])
    feat = []
    # feat = [yoe]


    if inc_year:
        feat += one_hot_year
    if inc_month:
        feat += one_hot_month
    if inc_day:
        feat += one_hot_day
    if inc_hour:
        feat += one_hot_hour

    # feat.append(yoe)
            
    return feat

In [97]:
def feature_combined(d):
    feat = []
    feat += featureBOW(d)
    feat += feature_time(d)
    feat += dma_yoe_feat(d, dmaindices)
    feat += feat_deg(d)
    return feat

In [98]:
X = [feature_combined(d) for d in X_train]
y = y_train[:]

In [99]:
# print(len(X[0]), X[0])

In [ ]:
# Training

In [103]:
mod = linear_model.Ridge(1)

mod.fit(X, y)

pred = mod.predict(X)


In [104]:
mse = sklearn.metrics.mean_squared_error(y, pred)
mae = sklearn.metrics.mean_absolute_error(y, pred)
mape = 100*sklearn.metrics.mean_absolute_percentage_error(y, pred)

print(math.sqrt(mse))
print(mae)
print(mape)

29483.711683454185
18973.920461890462
19.208924335992265


In [108]:
X = [feature_combined(d) for d in X_valid]
y = y_valid[:]

pred = mod.predict(X)

In [109]:
mse = sklearn.metrics.mean_squared_error(y, pred)
mae = sklearn.metrics.mean_absolute_error(y, pred)
mape = 100*sklearn.metrics.mean_absolute_percentage_error(y, pred)

print(math.sqrt(mse))
print(mae)
print(mape)

29672.192071812216
19342.75170095618
21.001993675115248
